In [16]:
import hashlib
import json
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import imageio
import time
from datetime import datetime

# فولدری برای ذخیره فریم‌ها بساز
FRAME_DIR = "frames"
os.makedirs(FRAME_DIR, exist_ok=True)

# تنظیمات اندازه بلوک‌ها
BLOCK_WIDTH = 300
BLOCK_HEIGHT = 110
BLOCK_GAP = 60  # فاصله بین بلوک‌ها

# رنگ‌ها برای وضعیت‌های مختلف بلوک
COLOR_PENDING = "#a6cee3"  # رنگ بلوک‌های در حال انتظار
COLOR_MINING = "#fdbf6f"   # رنگ بلوکی که در حال استخراج است
COLOR_MINED = "#33a02c"    # رنگ بلوک استخراج شده

# تابعی برای محاسبه هش SHA256 یک بلاک با توجه به فیلدهای آن
def calculate_hash(index, previous_hash, timestamp, data, nonce):
    block_string = json.dumps({
        "index": index,
        "previous_hash": previous_hash,
        "timestamp": timestamp,
        "data": data,
        "nonce": nonce
    }, sort_keys=True).encode()
    return hashlib.sha256(block_string).hexdigest()

# تبدیل timestamp به رشته قابل فهم (مثلا: '12:30:15')
def format_timestamp(ts):
    return datetime.fromtimestamp(ts).strftime('%H:%M:%S')

# تابعی برای رسم یک بلاک در مختصات مشخص روی محورهای matplotlib
def draw_block(ax, x, y, block, color, highlight_nonce=None):
    rect = patches.FancyBboxPatch((x, y), BLOCK_WIDTH, BLOCK_HEIGHT,
                                  boxstyle="round,pad=0.1",
                                  linewidth=2, edgecolor="black",
                                  facecolor=color)
    ax.add_patch(rect)

    # متن‌های داخل بلاک با اطلاعات کامل‌تر
    lines = [
        f"Index: {block['index']}",
        f"Nonce: {highlight_nonce if highlight_nonce is not None else block['nonce']}",
        f"Hash: {block['hash'][:15]}...",  # کوتاه شده هش
        f"Prev Hash: {block['previous_hash'][:15]}...",  # هش قبلی کوتاه شده
        f"Timestamp: {format_timestamp(block['timestamp'])}"
    ]
    for i, line in enumerate(lines):
        ax.text(x + 10, y + BLOCK_HEIGHT - 20 - i*18, line, fontsize=6, color="black", fontweight='bold')

# تابعی برای رسم فلش بین دو بلاک (از x_start,y_start به x_end,y_end)
def draw_arrow(ax, x_start, y_start, x_end, y_end):
    ax.annotate("",
                xy=(x_end, y_end + BLOCK_HEIGHT/2),
                xytext=(x_start + BLOCK_WIDTH, y_start + BLOCK_HEIGHT/2),
                arrowprops=dict(arrowstyle="->", lw=2))

# تابعی برای ذخیره هر فریم انیمیشن به عنوان تصویر PNG
def save_frame(fig, frame_num):
    filename = os.path.join(FRAME_DIR, f"frame_{frame_num:03d}.png")
    fig.savefig(filename)
    plt.close(fig)
    return filename

# تابع اصلی برای شبیه‌سازی استخراج بلاک و ساخت انیمیشن
def visualize_mining(difficulty=4, max_nonce=150000, num_blocks=5):
    # ساخت بلاک اولیه (Genesis)
    genesis_timestamp = time.time()
    genesis_block = {
        "index": 0,
        "previous_hash": "0",
        "timestamp": genesis_timestamp,
        "data": "Genesis Block",
        "nonce": 0,
        "hash": calculate_hash(0, "0", genesis_timestamp, "Genesis Block", 0)
    }
    blockchain = [genesis_block]

    frame_files = []
    frame_num = 0

    # تابع داخلی برای رسم کل زنجیره و بلاک فعلی
    def draw_scene(current_nonce=None, mining_block_index=None, mined_block_index=None):
        fig, ax = plt.subplots(figsize=(14, 4))
        ax.set_xlim(0, (BLOCK_WIDTH + BLOCK_GAP) * (len(blockchain) + 1))
        ax.set_ylim(0, BLOCK_HEIGHT + 20)
        ax.axis('off')

        # رسم بلاک‌های تایید شده
        for i, block in enumerate(blockchain):
            color = COLOR_MINED
            draw_block(ax, i * (BLOCK_WIDTH + BLOCK_GAP), 10, block, color)

        # رسم بلاک در حال استخراج
        if mining_block_index is not None:
            temp_block = {
                "index": mining_block_index,
                "previous_hash": blockchain[-1]["hash"],
                "timestamp": time.time(),
                "data": f"Block #{mining_block_index} Data",
                "nonce": current_nonce if current_nonce is not None else 0,
                "hash": calculate_hash(mining_block_index, blockchain[-1]["hash"],
                                       time.time(), f"Block #{mining_block_index} Data", current_nonce if current_nonce is not None else 0)
            }
            draw_block(ax, mining_block_index * (BLOCK_WIDTH + BLOCK_GAP), 10, temp_block, COLOR_MINING, highlight_nonce=current_nonce)

        # رسم فلش‌ها
        for i in range(len(blockchain) - 1):
            draw_arrow(ax, i * (BLOCK_WIDTH + BLOCK_GAP), 10, (i + 1) * (BLOCK_WIDTH + BLOCK_GAP), 10)
        if mining_block_index is not None:
            draw_arrow(ax, (mining_block_index - 1) * (BLOCK_WIDTH + BLOCK_GAP), 10, mining_block_index * (BLOCK_WIDTH + BLOCK_GAP), 10)

        filename = save_frame(fig, frame_num)
        return filename

    # فریم اولیه فقط با بلاک پیدایش
    frame_files.append(draw_scene(mined_block_index=0))
    frame_num += 1

    # استخراج به تعداد مشخص بلاک
    for block_num in range(1, num_blocks + 1):
        index = block_num
        previous_hash = blockchain[-1]["hash"]
        timestamp = time.time()
        data = f"Block #{block_num} Data"
        nonce = 0
        target = '0' * difficulty

        while nonce < max_nonce:
            current_hash = calculate_hash(index, previous_hash, timestamp, data, nonce)

            if nonce % 1000 == 0:
                frame_files.append(draw_scene(current_nonce=nonce, mining_block_index=index))
                frame_num += 1

            if current_hash.startswith(target):
                new_block = {
                    "index": index,
                    "previous_hash": previous_hash,
                    "timestamp": timestamp,
                    "data": data,
                    "nonce": nonce,
                    "hash": current_hash
                }
                blockchain.append(new_block)
                frame_files.append(draw_scene(mined_block_index=index))
                frame_num += 1

                print(f"✅ Block {index} mined! Nonce: {nonce}, Hash: {current_hash}")
                break

            nonce += 1

    # ساخت فایل GIF
    gif_path = "blockchain_mining.gif"
    with imageio.get_writer(gif_path, mode='I', duration=2.5) as writer:
        for filename in frame_files:
            image = imageio.imread(filename)
            writer.append_data(image)
    print(f"🎬 Mining animation saved as {gif_path}")

    # حذف فریم‌ها
    

if __name__ == "__main__":
    visualize_mining(difficulty=4)


✅ Block 1 mined! Nonce: 2718, Hash: 00009158d3a2ef7d57c4c94b2db2867012fbe218eeaf24ce7b5afcac055b668f
✅ Block 2 mined! Nonce: 12650, Hash: 00001d5d9b1cc2893c586766c56ba3855059fe9c83b2f71144db3669ea123e00
✅ Block 3 mined! Nonce: 28041, Hash: 0000d73c7c59ae3d9f5c49acdeb68c3dcd6cd6fbe434428fb9336c13fecb46e3
✅ Block 4 mined! Nonce: 8562, Hash: 00003f6e92fe5d1bf8baa1cadd051bc7bf0b335a6dec2c16afacf4258a7d821c
✅ Block 5 mined! Nonce: 31027, Hash: 000091a91eee1c519de726a340eca175380551270454d71d353ff05392da1ee0


C:\Users\Hossein\AppData\Local\Temp\ipykernel_3944\4019892543.py:165: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


🎬 Mining animation saved as blockchain_mining.gif


In [17]:
# کتابخانه‌های مورد استفاده برای رسم، ایجاد تصاویر و فایل‌های انیمیشن
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import imageio
import os
import random
import math

# مسیر ذخیره‌سازی فریم‌ها (تصاویر هر مرحله)
FRAME_DIR = "consensus_frames"
os.makedirs(FRAME_DIR, exist_ok=True)  # اگر فولدر وجود نداشت، ایجاد شود

NUM_NODES = 10            # تعداد نودهای شرکت‌کننده در اجماع
FRAME_DURATION = 20      # مدت زمان نمایش هر فریم در انیمیشن (برحسب ثانیه)

# محاسبه موقعیت نودها به صورت دایره‌ای (برای توزیع زیبا در نمودار)
def node_positions(num_nodes, radius=3):
    angle_gap = 2 * math.pi / num_nodes
    positions = []
    for i in range(num_nodes):
        angle = i * angle_gap
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        positions.append((x, y))
    return positions

# تابع رسم نودها (دایره با متن شماره نود داخل آن)
def draw_node(ax, x, y, node_id):
    circle = plt.Circle((x, y), 0.3, color='skyblue', ec='black', lw=2)
    ax.add_patch(circle)
    ax.text(x, y, f"N{node_id}", fontsize=12, ha='center', va='center', fontweight='bold')

# رسم بلاک پیشنهادی کنار نود
def draw_block(ax, x, y, proposer_id, block_label, color):
    rect = patches.FancyBboxPatch((x, y), 1.5, 0.7,
                                  boxstyle="round,pad=0.1",
                                  linewidth=2, edgecolor="black",
                                  facecolor=color)
    ax.add_patch(rect)
    ax.text(x + 0.75, y + 0.35, f"P{proposer_id}\n{block_label}", ha='center', va='center', fontsize=10, fontweight='bold')

# رسم فلش رأی‌دادن از یک نود به بلاک پیشنهادی
def draw_vote_arrow(ax, from_pos, to_pos):
    ax.annotate("",
                xy=to_pos,
                xytext=from_pos,
                arrowprops=dict(arrowstyle="->", lw=1.5, color="orange"))

# ذخیره فریم در فایل تصویر و بستن آن
def save_frame(fig, frame_num):
    filename = os.path.join(FRAME_DIR, f"frame_{frame_num:03d}.png")
    fig.savefig(filename)
    plt.close(fig)
    return filename

# شبیه‌سازی یک دور اجماع
def simulate_consensus_round(nodes, frame_start_num):
    positions = node_positions(len(nodes))  # موقعیت نودها
    frame_files = []  # لیست فایل‌های تصویری تولید شده
    frame_num = frame_start_num

    # تولید بلاک‌های پیشنهادی توسط هر نود (هش جعلی برای شبیه‌سازی)
    proposals = []
    for node_id in nodes:
        block_hash = f"{random.randint(1000,9999)}abcd"
        proposals.append({"proposer": node_id, "hash": block_hash})

    # ✅ فریم اول: نمایش نودها و بلاک‌های پیشنهادی‌شان
    fig, ax = plt.subplots(figsize=(8,8))
    ax.set_xlim(-5,5)
    ax.set_ylim(-5,5)
    ax.axis('off')  # حذف محورهای نمودار

    for (x,y), node_id in zip(positions, nodes):
        draw_node(ax, x, y, node_id)  # رسم نودها

    for (x,y), prop in zip(positions, proposals):
        draw_block(ax, x-0.75, y+0.5, prop["proposer"], f"Block {prop['hash']}", color="#87ceeb")  # رسم بلاک‌ها

    ax.set_title("Nodes and Their Proposed Blocks", fontsize=16)
    frame_files.append(save_frame(fig, frame_num))
    frame_num += 1

    # ✅ رأی‌دادن: هر نود به‌صورت تصادفی به یکی از بلاک‌ها رأی می‌دهد
    votes = []
    for voter_id in nodes:
        chosen_block = random.choice(proposals)
        votes.append({"voter": voter_id, "vote_for": chosen_block})

    # ✅ فریم‌های مربوط به نمایش پویای رأی‌دادن
    fig, ax = plt.subplots(figsize=(8,8))
    ax.set_xlim(-5,5)
    ax.set_ylim(-5,5)
    ax.axis('off')

    for (x,y), node_id in zip(positions, nodes):
        draw_node(ax, x, y, node_id)

    for (x,y), prop in zip(positions, proposals):
        draw_block(ax, x-0.75, y+0.5, prop["proposer"], f"Block {prop['hash']}", color="#87ceeb")

    # فریم به‌فریم رأی‌دادن را با فلش نمایش می‌دهیم
    for i, vote in enumerate(votes):
        voter_pos = positions[vote["voter"]]
        voted_prop = vote["vote_for"]
        prop_index = next(i for i, p in enumerate(proposals) if p == voted_prop)
        block_pos = (positions[prop_index][0], positions[prop_index][1] + 0.85)

        for j in range(i+1):  # نمایش تمام فلش‌های قبلی تا این لحظه
            voter_pos_j = positions[votes[j]["voter"]]
            voted_prop_j = votes[j]["vote_for"]
            prop_index_j = next(idx for idx,p in enumerate(proposals) if p == voted_prop_j)
            block_pos_j = (positions[prop_index_j][0], positions[prop_index_j][1] + 0.85)
            draw_vote_arrow(ax, voter_pos_j, block_pos_j)

        ax.set_title(f"Voting in Progress: Vote {i+1}/{len(votes)}", fontsize=16)
        frame_files.append(save_frame(fig, frame_num))
        frame_num += 1

    # ✅ شمارش آرا
    vote_counts = {}
    for v in votes:
        key = v["vote_for"]["hash"]
        vote_counts[key] = vote_counts.get(key, 0) + 1

    # پیدا کردن بلاکی که بیشترین رأی را دارد
    winner_hash = max(vote_counts, key=vote_counts.get)
    winner_prop = next(p for p in proposals if p["hash"] == winner_hash)
    winner_votes = vote_counts[winner_hash]

    # ✅ فریم نهایی: نمایش بلاک برنده با رنگ سبز و باقی با خاکستری
    fig, ax = plt.subplots(figsize=(8,8))
    ax.set_xlim(-5,5)
    ax.set_ylim(-5,5)
    ax.axis('off')

    for (x,y), node_id in zip(positions, nodes):
        draw_node(ax, x, y, node_id)

    for (x,y), prop in zip(positions, proposals):
        color = "#33a02c" if prop == winner_prop else "#b0b0b0"
        draw_block(ax, x-0.75, y+0.5, prop["proposer"], f"Block {prop['hash']}", color=color)

    # رسم تمام فلش‌های رأی‌دهی
    for vote in votes:
        voter_pos = positions[vote["voter"]]
        voted_prop = vote["vote_for"]
        prop_index = next(i for i, p in enumerate(proposals) if p == voted_prop)
        block_pos = (positions[prop_index][0], positions[prop_index][1] + 0.85)
        draw_vote_arrow(ax, voter_pos, block_pos)

    ax.set_title(f"Consensus Reached: Node {winner_prop['proposer']}'s Block Wins with {winner_votes} Votes", fontsize=16, color='green')
    frame_files.append(save_frame(fig, frame_num))
    frame_num += 1

    return frame_files, frame_num  # بازگرداندن لیست فریم‌ها و شماره فریم بعدی

# ✅ ایجاد انیمیشن از چندین دور اجماع
def create_consensus_gif(total_rounds=3):
    all_frames = []
    frame_num = 0
    nodes = list(range(NUM_NODES))  # نودهای 0 تا NUM_NODES - 1

    for r in range(total_rounds):
        print(f"Simulating consensus round {r+1}...")
        frames, frame_num = simulate_consensus_round(nodes, frame_num)
        all_frames.extend(frames)

    # تولید فایل GIF از تمام فریم‌ها
    gif_path = "consensus_simulation.gif"
    with imageio.get_writer(gif_path, mode='I', duration=FRAME_DURATION) as writer:
        for filename in all_frames:
            image = imageio.imread(filename)
            writer.append_data(image)

    print(f"Consensus voting animation saved as {gif_path}")

    # پاک کردن فایل‌های فریم موقتی
    

# اجرای اصلی برنامه
if __name__ == "__main__":
    create_consensus_gif(total_rounds=10)  # تعداد دورهای اجماع: ۳


Simulating consensus round 1...
Simulating consensus round 2...
Simulating consensus round 3...
Simulating consensus round 4...
Simulating consensus round 5...
Simulating consensus round 6...
Simulating consensus round 7...
Simulating consensus round 8...
Simulating consensus round 9...
Simulating consensus round 10...


C:\Users\Hossein\AppData\Local\Temp\ipykernel_3944\2659392276.py:173: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


Consensus voting animation saved as consensus_simulation.gif
